# Partie A : Exploration des données 

In [18]:
# Importation des bibliothèques nécessaires
import pandas as pd


# 1.  Charger le fichier CSV ou Excel avec pandas. 
df = pd.read_csv("../data/ventes.csv")


# 2.  Afficher les 5 premières lignes du dataset. 
print(df.head())

# 3.  Afficher les informations générales du dataset 
print("\n------------------------------------------")
print("\nLes informations générales du dataset :"
      , df.info()
      )

print("\n------------------------------------------")
print("\nLes statistiques descriptives du dataset :"
      ,df.describe()
      )

# 4.  Identifier s’il existe des valeurs manquantes. 
print("\n------------------------------------------")
print("\nLa liste des produits qui ont des valeurs manquantes : \n"
      ,df.isnull().sum()
      )


         Date     Produit     Catégorie  Quantité  PrixUnitaire    Magasin
0  2022-04-13  Smartphone          Mode        10       1009.00       Lyon
1  2023-03-12    Pantalon  Électronique         2        782.86      Paris
2  2022-09-28    Pantalon          Mode        19       1159.62   Bordeaux
3  2022-04-17     Fromage  Électronique         9        782.64   Bordeaux
4  2022-03-13          TV          Mode         7       1279.01  Marseille

------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          200 non-null    object 
 1   Produit       200 non-null    object 
 2   Catégorie     200 non-null    object 
 3   Quantité      200 non-null    int64  
 4   PrixUnitaire  200 non-null    float64
 5   Magasin       200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+

# Partie B : Nettoyage et préparation 

In [ ]:
# 5.  Convertir la colonne Date en type datetime.

# print(df.head())
# df['Date'] = pd.to_datetime(df['Date'])
# print("--------------------------------------------------------------------------")
# print(df['Date'].dtypes)
# df.to_csv("../data/ventes.csv")
# print("\n--------------------------------------------------------------------------")
# print(df.head())


# 6.  Créer une nouvelle colonne ChiffreAffaires = Quantité * PrixUnitaire.

# print("\n--------------------------------------------------------------------------")
# df['ChiffreAffaires'] = df['Quantité'] * df['PrixUnitaire']

# 7.  Vérifier s’il existe des doublons et les supprimer.

print("\n--------------------------------------------------------------------------")

# df.loc[len(df)] = [pd.to_datetime('2022-04-13'), 'Smartphone', 'Mode', 10, 1009.00, 'Lyon', 10090.00]
print("\nListe des produits doublons : \n",df.duplicated())
print("\nNombre de doublons sur 'Produit' : ", df.duplicated().sum())
df.drop_duplicates(inplace=True)
print("\nNombre de doublons sur 'Produit' : ", df.duplicated().sum())



--------------------------------------------------------------------------

Liste des produits doublons : 
 0     False
1     False
3     False
4     False
6     False
10    False
dtype: bool

Nombre de doublons sur 'Produit' :  0

Nombre de doublons sur 'Produit' :  0


# Partie C : Analyses simples 

In [99]:
# 8.  Trouver le produit le plus vendu en termes de quantité. 

print("\nProduit le plus vendu : \n",df.loc[df['Quantité'].idxmax()])
print("****************************************")
print("\nLe produit le plus cher : \n",df.loc[df['PrixUnitaire'].idxmax()])

print("\n-----------------------------------------------")

print("\nProduit le moins vendu : \n",df.loc[df['Quantité'].idxmin()])
print("*****************************************")
print("\nProduit le moins cher : \n",df.loc[df['PrixUnitaire'].idxmin()])


# 9.  Calculer le chiffre d’affaires total par catégorie. 
print("\n-----------------------------------------------")

chiff_par_categorie = df.groupby('Catégorie', as_index=False)['ChiffreAffaires'].sum()

print("\n",chiff_par_categorie)


Produit le plus vendu : 
 Date               2022-01-21 00:00:00
Produit                        T-shirt
Catégorie                 Électronique
Quantité                            18
PrixUnitaire                     843.6
Magasin                      Marseille
ChiffreAffaires                15184.8
Name: 6, dtype: object
****************************************

Le produit le plus cher : 
 Date               2022-03-13 00:00:00
Produit                             TV
Catégorie                         Mode
Quantité                             7
PrixUnitaire                   1279.01
Magasin                      Marseille
ChiffreAffaires                8953.07
Name: 4, dtype: object

-----------------------------------------------

Produit le moins vendu : 
 Date               2023-03-12 00:00:00
Produit                       Pantalon
Catégorie                 Électronique
Quantité                             2
PrixUnitaire                    782.86
Magasin                          Paris


In [100]:
pivot = df.pivot_table(
    values= 'ChiffreAffaires',
    index= 'Catégorie',
    aggfunc= 'sum'
)
pivot

,ChiffreAffaires
Catégorie,
Mode,19235.19
Électronique,23794.28
